In [1]:
from datasets import load_dataset

ds = load_dataset("jhu-clsp/jfleg")

README.md: 0.00B [00:00, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/141k [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/755 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/748 [00:00<?, ? examples/s]

In [4]:
ds["test"].head()

AttributeError: 'Dataset' object has no attribute 'head'